In [58]:
import pandas as pd
import numpy as np
import string
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [59]:
def extract_csv_from_folder(folder_name: str) -> dict[str, str]:
    for root, dirs, files in os.walk(folder_name, topdown=False):
        df_paths: dict = {file: f'{root}/{file}' for file in files if file.endswith('csv')}
    return df_paths

In [60]:
df_paths = extract_csv_from_folder('competitive-data-science-predict-future-sales/raw')
df_paths

{'sales_train.csv': 'competitive-data-science-predict-future-sales/raw/sales_train.csv',
 'shops.csv': 'competitive-data-science-predict-future-sales/raw/shops.csv',
 'test.csv': 'competitive-data-science-predict-future-sales/raw/test.csv',
 'item_categories.csv': 'competitive-data-science-predict-future-sales/raw/item_categories.csv',
 'items.csv': 'competitive-data-science-predict-future-sales/raw/items.csv',
 'sample_submission.csv': 'competitive-data-science-predict-future-sales/raw/sample_submission.csv'}

In [61]:
def load_data(df_paths: dict[str, str]) -> dict[str, pd.DataFrame]:
    data: dict[str, pd.DataFrame] = {}
    
    for filename, path in df_paths.items():
        data[filename] = pd.read_csv(path)
        
    return data

In [62]:
data = load_data(df_paths)

In [63]:
sales_train = data['sales_train.csv'].copy()
shops = data['shops.csv'].copy()
test = data['test.csv'].copy()
item_categories = data['item_categories.csv'].copy()
items = data['items.csv'].copy()

In [64]:
del data

Perform the same filtrations as we did in the DQC

In [67]:
sales_train.date = pd.to_datetime(sales_train.date)

In [68]:
sales_train['month'] = sales_train.date.dt.to_period('M')

In [69]:
sales_train = sales_train[sales_train.item_cnt_day > -2]

In [70]:
sales_train = sales_train[sales_train.month < '2015-11']

In [71]:
shops = shops[shops.shop_id != 10]

In [72]:
sales_train.loc[sales_train.shop_id == 10, 'shop_id'] = 11

In [73]:
shops.shop_name = shops.shop_name.map(lambda x: x.lstrip('!'))

In [74]:
shops[['city', 'shop_name']] = shops['shop_name'].str.split(' ', n=1, expand=True)

In [75]:
sales_train = sales_train.drop_duplicates(
    subset=['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_day'], 
    keep='last')

In [76]:
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [77]:
items.item_name = items.item_name.map(lambda x: x.lstrip('!*/'))

In [78]:
same_items_regargless_case = items[items.item_name.map(lambda x: x.lower()).duplicated()]

In [79]:
fully_same = items.groupby(items.item_name, as_index=False).size().query('size > 1')

In [80]:
fully_same_item_names = fully_same.item_name.values

In [81]:
fully_same_item_ids = items[items.item_name.isin(fully_same_item_names)].item_id.values

In [82]:
items = items[items.item_id != 12]

In [83]:
sales_train = sales_train[sales_train.item_id != 12]

In [84]:
same_items_regargless_case_names = same_items_regargless_case[~same_items_regargless_case.item_name.isin(
    fully_same_item_names)].item_name

In [85]:
same_items = items[items.item_name.map(lambda x: x.lower()).isin(
                same_items_regargless_case_names.map(lambda x: x.lower()))]

In [86]:
same_items_upper_case = same_items[same_items.item_name.str.endswith('(Регион)')]
same_items_lower_case = same_items[same_items.item_name.str.endswith('(регион)')]

In [87]:
assert (len(same_items_upper_case) + len(same_items_lower_case) == len(same_items))

In [88]:
shops_upper_case = sales_train[sales_train.item_id.isin(same_items_upper_case.item_id)].shop_id.unique()
shops_lower_case = sales_train[sales_train.item_id.isin(same_items_lower_case.item_id)].shop_id.unique()

shops_upper_case.sort()
shops_lower_case.sort()

In [89]:
assert (np.array_equal(shops_upper_case, shops_lower_case))

In [90]:
assert (same_items_upper_case.item_id.values == same_items_lower_case.item_id.map(lambda x: x - 1).values).all()

In [91]:
items = items[~items.item_id.isin(same_items_lower_case.item_id)]

In [92]:
sales_train.loc[sales_train.item_id.isin(same_items_lower_case.item_id), 'item_id'] = \
    sales_train[sales_train.item_id.isin(same_items_lower_case.item_id)].item_id.map(lambda x: x - 1)

In [93]:
test.loc[test.item_id.isin(same_items_lower_case.item_id), 'item_id'] = \
    test[test.item_id.isin(same_items_lower_case.item_id)].item_id.map(lambda x: x - 1)

In [94]:
items = items[items.item_id != 13012]

In [95]:
sales_train.loc[sales_train.item_id == 13012, 'item_id'] = 13011
test.loc[test.item_id == 13012, 'item_id'] = 13011

In [96]:
sales_train = pd.merge(sales_train, items, on='item_id', how='inner')
sales_train = pd.merge(sales_train, item_categories, on='item_category_id', how='inner')

In [97]:
sales_train = pd.merge(sales_train, shops, on='shop_id', how='inner')

In [98]:
sales_train.isna().sum()

date                  0
date_block_num        0
shop_id               0
item_id               0
item_price            0
item_cnt_day          0
month                 0
item_name             0
item_category_id      0
item_category_name    0
shop_name             0
city                  0
dtype: int64

In [100]:
common_path = 'competitive-data-science-predict-future-sales/preprocessed'

In [103]:
if not os.path.exists(common_path):
    os.mkdir(common_path)

In [106]:
sales_train.to_csv(f'{common_path}/sales_train.csv')
shops.to_csv(f'{common_path}/shops.csv')
test.to_csv(f'{common_path}/test.csv')
item_categories.to_csv(f'{common_path}/item_categories.csv')
items.to_csv(f'{common_path}/items.csv')